In [1]:
import IJulia
import Base64

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie

  # CairoMakie's display() in PDF format opens an interactive window
  # instead of saving to the ipynb file, so we don't do that.
  # https://github.com/quarto-dev/quarto-cli/issues/7548
  if fig_format == :pdf
    CairoMakie.activate!(type = "png")
  else
    CairoMakie.activate!(type = string(fig_format))
  end
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = "L3J1bi9tZWRpYS9hbGYvZGF0b3MvbWlzcmVwb3NpdG9yaW9zL2RvY2VuY2lhL2FwcmVuZGl6YWplLWF1dG9tYXRpY28tcHJhY3RpY2FzLWp1bGlh"
  if !isempty(run_path)
    run_path = String(Base64.base64decode(run_path))
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
function fibonacci(n::Int)
    a = zeros(Float32, n)
    a[1] = 1
    a[2] = 1
    for i in 3:n
        a[i] = a[i-1] + a[i-2]
    end
    return a
end

fibonacci (generic function with 1 method)

In [3]:
fib = fibonacci(30) 
ventana = 4
n = 30 - ventana
# La red esperar una entrada con dimensiones (características, ventana, lote).
X =  Array{Float32}(undef, 1, ventana, n)
Y = Array{Float32}(undef, 1, n)  
for i = 1:n
    serie = fib[i:i+ventana-1]
    etiqueta = fib[i+ventana]
    X[1, : , i] .= serie
    Y[1, i] = etiqueta
end

In [4]:
using Lux, Random

modelo = Chain(
    Recurrence(RNNCell(1 => 8, identity); return_sequence = false),
    Dense(8 => 1)
)

rng = Random.default_rng()
ps, st = Lux.setup(rng, modelo)

┌ Warning: `replicate` doesn't work for `TaskLocalRNG`. Returning the same `TaskLocalRNG`.
└ @ LuxCore ~/.julia/packages/LuxCore/qsnGJ/src/LuxCore.jl:18


((layer_1 = (weight_ih = Float32[-0.7426836; 0.12309409; … ; -0.28318766; -0.61774004;;], weight_hh = Float32[-1.3849927 -1.2179685 … -0.20003402 0.27753624; -0.84857047 0.20263164 … 1.7670047 -0.948069; … ; 0.65203416 -0.15630409 … 1.2111578 -0.105777234; -1.7353704 0.13904727 … 0.4442589 -0.22340277], bias_ih = Float32[-1.2252676, -0.7584686, 0.10875261, -1.1166816, 0.11630381, 0.26449895, -0.45721933, 0.1870894], bias_hh = Float32[-1.0104553, -0.47301367, -0.5633451, -1.5514491, 0.38915062, 0.5585324, -0.95478314, -0.9064933]), layer_2 = (weight = Float32[0.5049418 0.12618382 … 0.24612837 0.060075313], bias = Float32[-0.13585447])), (layer_1 = (rng = TaskLocalRNG(),), layer_2 = NamedTuple()))

In [5]:
using Statistics
function coste(ps, st, X, Y)
    ŷ, _ = modelo(X, ps, st)          # ŷ: (1, batch)
    return mean((ŷ .- Y).^2)
end

coste(ps, st, X, Y)

2.0645175f11

In [6]:
using Optimisers, Zygote
opt = Optimisers.setup(Optimisers.Adam(0.01f0), ps)

nepocas = 200
for epoca in 1:nepocas
    # gradiente de la red con respecto a los parámetros
    gs = first(Zygote.gradient(p -> coste(p, st, X, Y), ps))
    opt, ps = Optimisers.update(opt, ps, gs) 
    println("Época $epoca | coste = ", coste(ps, st, X, Y))
end

Época 1 | coste = 1.3512544e11


Época 2 | coste = 8.325458e10
Época 3 | coste = 4.7182803e10
Época 4 | coste = 2.356902e10
Época 5 | coste = 9.449397e9
Época 6 | coste = 2.3006546e9
Época 7 | coste = 3.0115438e7
Época 8 | coste = 9.2948243e8
Época 9 | coste = 3.6217265e9
Época 10 | coste = 7.026311e9
Época 11 | coste = 1.0349568e10
Época 12 | coste = 1.3077052e10
Época 13 | coste = 1.4942196e10
Época 14 | coste = 1.5871922e10
Época 15 | coste = 1.5925694e10
Época 16 | coste = 1.5241261e10
Época 17 | coste = 1.3992682e10
Época 18 | coste = 1.2361161e10
Época 19 | coste = 1.0516543e10
Época 20 | coste = 8.60681e9
Época 21 | coste = 6.753225e9
Época 22 | coste = 5.0488786e9
Época 23 | coste = 3.5595922e9
Época 24 | coste = 2.3259228e9
Época 25 | coste = 1.3658797e9
Época 26 | coste = 6.779652e8
Época 27 | coste = 2.4445414e8
Época 28 | coste = 3.480324e7
Época 29 | coste = 9.223796e6
Época 30 | coste = 1.2235476e8
Época 31 | coste = 3.2695264e8
Época 32 | coste = 5.7742144e8
Época 33 | coste = 8.3297165e8
Época 34 | co

In [7]:
X_test = reshape(fib[end-ventana : end-1], 1, ventana, 1) 

y_test, _ = modelo(X_test, ps, st)
println("Predicción del término 30: ",  y_test[1, 1])

Predicción del término 30: 832045.25